In [9]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import math
from collections import Counter

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

### Data Dictionary
Variable	Definition	Key
survival	Survival	0 = No, 1 = Yes
pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	Sex	
Age	Age in years	
sibsp	# of siblings / spouses aboard the Titanic	
parch	# of parents / children aboard the Titanic	
ticket	Ticket number	
fare	Passenger fare	
cabin	Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton
Variable Notes
pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

In [10]:
train_df = pd.read_csv('/home/mathewlech/sandbox_local/input/titanic/train.csv')
test_df = pd.read_csv('/home/mathewlech/sandbox_local/input/titanic/test.csv')
combine = [train_df, test_df]
transformed_Sex = False
transformed_Embark = False

age_buckets = 20
fare_buckets = 10

#train_df.info()
print("--"*40)
#test_df.info()

--------------------------------------------------------------------------------


In [11]:
# Outlier detection 
def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detect outliers from Age, SibSp , Parch and Fare
Outliers_to_drop = detect_outliers(train_df,2,["Age","SibSp","Parch","Fare"])

/home/mathewlech/anaconda3/envs/primary_env/lib/python3.6/site-packages/numpy/lib/function_base.py:3652: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [12]:
train_df.loc[Outliers_to_drop]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.00,C23 C25 C27,S
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.00,C23 C25 C27,S
159,160,0,3,"Sage, Master. Thomas Henry",male,NaN,8,2,CA. 2343,69.55,NaN,S
180,181,0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,CA. 2343,69.55,NaN,S
201,202,0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.55,NaN,S
324,325,0,3,"Sage, Mr. George John Jr",male,NaN,8,2,CA. 2343,69.55,NaN,S
341,342,1,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.00,C23 C25 C27,S
792,793,0,3,"Sage, Miss. Stella Anna",female,NaN,8,2,CA. 2343,69.55,NaN,S
846,847,0,3,"Sage, Mr. Douglas Bullen",male,NaN,8,2,CA. 2343,69.55,NaN,S
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.55,NaN,S


In [13]:
# Drop outliers
train_df = train_df.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

* 77% of cabin is null probably not much use for this column
* passengerID doesn't intrinsically give us any real information
* Ticket doesnt seemt to provide insight


### Data manipulations

In [14]:
if 'Cabin' in train_df:
    train_df = train_df.drop(['Cabin'], axis=1)
if 'Cabin' in test_df:
    test_df = test_df.drop(['Cabin'], axis=1)       
    
if 'Ticket' in train_df:
    train_df = train_df.drop(['Ticket'], axis=1)
if 'Ticket' in test_df:
    test_df = test_df.drop(['Ticket'], axis=1)    
    
combine = [train_df, test_df]    

In [15]:
freq_port = train_df.Embarked.dropna().mode()[0]

In [16]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int) 

In [17]:
train_df['FamilySize']  = 0
test_df['FamilySize']  = 0
combine = [train_df, test_df]
for dataset in combine:        
        dataset['FamilySize'] = (dataset['Parch'] + dataset['SibSp']).astype(int) + 1        

In [18]:
train_df['IsAlone'] = 0
test_df['IsAlone'] = 0
train_df.loc[ (train_df.FamilySize == 1), 'IsAlone'] = 1
test_df.loc[ (train_df.FamilySize == 1), 'IsAlone'] = 1

if 'Parch' in test_df:
    test_df = test_df.drop(['Parch'], axis=1)  
if 'SibSp' in test_df:
    test_df = test_df.drop(['SibSp'], axis=1)      

if 'Parch' in train_df:
    train_df = train_df.drop(['Parch'], axis=1)  
if 'SibSp' in train_df:
    train_df = train_df.drop(['SibSp'], axis=1)      

if 'FamilySize' in train_df:
    train_df = train_df.drop(['FamilySize'], axis=1)  
    
if 'FamilySize' in test_df:
    test_df = test_df.drop(['FamilySize'], axis=1)   
    
combine = [train_df, test_df]    

In [19]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)

In [20]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [21]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
combine = [train_df, test_df] 

### Age band Feature

In [22]:
guess_ages = np.zeros((2,3))
guess_ages

array([[0., 0., 0.],
       [0., 0., 0.]])

In [23]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

In [24]:
train_df['AgeBand'] = pd.cut(train_df['Age'], age_buckets,include_lowest=True).astype(str).str.strip('()[]')
age_band_buckets = train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)
age_band_buckets = age_band_buckets['AgeBand'].str.split(',', expand=True)
age_band_buckets.rename(columns={ age_band_buckets.columns[0]: "Min" , age_band_buckets.columns[1]: "Max"}, inplace=True)
age_band_buckets = age_band_buckets.dropna()
age_band_buckets.Min = age_band_buckets.Min.astype(float)
age_band_buckets.Max = age_band_buckets.Max.astype(float)
age_band_buckets.sort_values(by='Min', ascending=True,inplace=True)
#age_band_buckets.head(100).sort_values(by='Min', ascending=True)

In [25]:
train_df['AgeInt'] = -1
test_df['AgeInt'] = -1
combine = [train_df, test_df] 
a_rowcount = age_band_buckets.shape[0] -1
a_iterplace = 0
for index, row in age_band_buckets.sort_values(by='Min', ascending=True).iterrows():
    minval =  row['Min']
    maxval = row['Max']
    
    if a_iterplace == a_rowcount:#need to handle max value truncation        
        maxval += 1        
        
    a_iterplace +=1
    for dataset in combine:                         
        dataset.loc[(dataset['Age'] >= minval) & (dataset['Age'] <= maxval), 'AgeInt'] = index        

### Fare band feature

In [26]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], fare_buckets).astype(str).str.strip('()[]')
Fare_band_buckets = train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)
Fare_band_buckets = Fare_band_buckets['FareBand'].str.split(',', expand=True)
Fare_band_buckets.rename(columns={ Fare_band_buckets.columns[0]: "Min" , Fare_band_buckets.columns[1]: "Max"}, inplace=True)
Fare_band_buckets = Fare_band_buckets.dropna()
Fare_band_buckets.Min = Fare_band_buckets.Min.astype(float)
Fare_band_buckets.Max = Fare_band_buckets.Max.astype(float)
Fare_band_buckets.sort_values(by='Min', ascending=True,inplace=True)
#Fare_band_buckets.head(100).sort_values(by='Min', ascending=True)

In [27]:
train_df['FareInt'] = -1
test_df['FareInt'] = -1
combine = [train_df, test_df] 
f_rowcount = Fare_band_buckets.shape[0] -1
f_iterplace = 0
for index, row in Fare_band_buckets.sort_values(by='Min', ascending=True).iterrows():
    minval =  row['Min']
    maxval = row['Max']
    
    if f_iterplace == f_rowcount:#need to handle max value truncation        
        maxval += 1        
        
    f_iterplace +=1
    for dataset in combine:                         
        dataset.loc[(dataset['Fare'] >= minval) & (dataset['Fare'] <= maxval), 'FareInt'] = index        

In [28]:
if 'FareBand' in test_df:
    test_df = test_df.drop(['FareBand'], axis=1)  
if 'SibSp' in test_df:
    test_df = test_df.drop(['SibSp'], axis=1)      

if 'FareBand' in train_df:
    train_df = train_df.drop(['FareBand'], axis=1)  
if 'AgeBand' in train_df:
    train_df = train_df.drop(['AgeBand'], axis=1)    
    
if 'Fare' in train_df:
    train_df = train_df.drop(['Fare'], axis=1)  
if 'Age' in train_df:
    train_df = train_df.drop(['Age'], axis=1)   
    
if 'Fare' in test_df:
    test_df = test_df.drop(['Fare'], axis=1)  
if 'Age' in test_df:
    test_df = test_df.drop(['Age'], axis=1)    
    
combine = [train_df, test_df] 

### use name to generate a title

In [29]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

train_df['Title'].value_counts() 

Mr        513
Miss      180
Mrs       126
Master     39
Rare       23
Name: Title, dtype: int64

In [30]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [31]:
if 'Name' in test_df:
    test_df = test_df.drop(['Name'], axis=1)  
if 'Name' in train_df:
    train_df = train_df.drop(['Name'], axis=1)    
    
combine = [train_df, test_df] 

In [32]:
#train_df.info()
print("--"*40)
#test_df.info()

--------------------------------------------------------------------------------


In [33]:
#colormap = plt.cm.RdBu
#plt.figure(figsize=(14,12))
#plt.title('Pearson Correlation of Features', y=1.05, size=15)
#sns.heatmap(train_df[["AgeInt", "Pclass","FareInt","Survived","Sex",'IsAlone','Title']].astype(float).corr(),linewidths=0.1,vmax=1.0, 
#            square=True, cmap=colormap, linecolor='white', annot=True)

In [34]:
test_df.head()

,PassengerId,Pclass,Sex,Embarked,IsAlone,AgeInt,FareInt,Title
0,892,3,0,2,0,6,6,1
1,893,3,1,0,0,10,0,3
2,894,2,0,2,1,14,9,1
3,895,3,0,0,0,4,9,1
4,896,3,1,0,1,3,1,3


## Machine Learning

In [35]:
X_train = train_df.drop("Survived", axis=1)
X_train = X_train.drop("PassengerId", axis=1)
Y_train = train_df["Survived"]

X_test = test_df
X_test = X_test.drop("PassengerId", axis=1)

X_train.shape, Y_train.shape, X_test.shape

((881, 7), (881,), (418, 7))

In [36]:
mlp = MLPClassifier(solver='lbfgs', alpha=1e-3,
                     hidden_layer_sizes=(5, 2), random_state=1)

logreg = LogisticRegression()
ridge = RidgeClassifier()
svc = SVC()
knn = KNeighborsClassifier(n_neighbors = 3)
perceptron = Perceptron(alpha=1e-5)
gaussian = GaussianNB()
linear_svc = LinearSVC()
sgd = SGDClassifier()
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier(n_estimators=100)

models ={}
models["logreg"] = logreg
models["ridge"] = ridge
models["svc"] = svc
models["knn"] = knn
models["perceptron"] = perceptron
models["mlp"] = mlp
models["gaussian"] = gaussian
models["linear_svc"] = linear_svc
models["sgd"] = sgd
models["decision_tree"] = decision_tree
models["random_forest"] = random_forest

In [37]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import cross_val_score
scores_df = pd.DataFrame()
#allcoeffs = pd.DataFrame(columns=['Feature','Model','Correlation'])
for name, model in models.items():
    model.fit(X_train, Y_train)
    X_Pred = model.predict(X_train)
        
    sc = round(model.score(X_train, Y_train) * 100, 2)    
    precision, recall, fscore, support = score(Y_train, X_Pred)

    #coeff_df = pd.DataFrame(X_train.columns.delete(0))
    #coeff_df['Model'] = name
    #coeff_df.rename(columns={ coeff_df.columns[0]: "Feature" }, inplace=True)
    #coeff_df["Correlation"] = pd.Series(model.coef_[0])    
    
    #allcoeffs = allcoeffs.append(coeff_df)
    
    cvscore = cross_val_score(model, X_train, Y_train,
                                scoring='accuracy', cv=4)
    
    cv_mean = round(cvscore.mean() * 100, 2)   
    cv_stddev = round(cvscore.std() * 100 * 2, 2)   
    scores_df = scores_df.append({
                                    'Model':name, 
                                    'Score':sc,
                                    'CV_Mean': cv_mean,
                                    'CV_SDEV': cv_stddev,
                                    'Precision': precision,
                                    'recall': recall,
                                    'fscore': fscore,
                                    'support': support                                    
                                
    }, ignore_index=True)

/home/mathewlech/anaconda3/envs/primary_env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/mathewlech/anaconda3/envs/primary_env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning

In [38]:
scores_df = scores_df[['Model', 'Score', 'CV_Mean','CV_SDEV', 'Precision','recall','fscore','support',]]
scores_df.sort_values(by='Score', ascending=False,inplace=True)
scores_df.head(100)

,Model,Score,CV_Mean,CV_SDEV,Precision,recall,fscore,support
9,decision_tree,92.74,80.82,3.18,"[0.9090909090909091, 0.9630872483221476]","[0.9796672828096118, 0.8441176470588235]","[0.9430604982206405, 0.8996865203761755]","[541, 340]"
10,random_forest,92.74,82.29,2.19,"[0.9176882661996497, 0.9451612903225807]","[0.9685767097966729, 0.861764705882353]","[0.9424460431654677, 0.9015384615384615]","[541, 340]"
3,knn,87.29,77.76,3.20,"[0.8921389396709324, 0.8413173652694611]","[0.9020332717190388, 0.8264705882352941]","[0.8970588235294117, 0.8338278931750741]","[541, 340]"
2,svc,86.72,81.39,6.09,"[0.8693379790940766, 0.8631921824104235]","[0.922365988909427, 0.7794117647058824]","[0.895067264573991, 0.8191653786707882]","[541, 340]"
5,mlp,82.52,80.71,4.62,"[0.8230383973288815, 0.8297872340425532]","[0.911275415896488, 0.6882352941176471]","[0.8649122807017544, 0.7524115755627009]","[541, 340]"
1,ridge,80.02,79.91,2.77,"[0.8288288288288288, 0.7515337423312883]","[0.8502772643253235, 0.7205882352941176]","[0.8394160583941606, 0.7357357357357357]","[541, 340]"
7,linear_svc,80.02,79.80,2.70,"[0.8288288288288288, 0.7515337423312883]","[0.8502772643253235, 0.7205882352941176]","[0.8394160583941606, 0.7357357357357357]","[541, 340]"
8,sgd,79.91,62.32,28.29,"[0.825, 0.7538940809968847]","[0.8539741219963032, 0.711764705882353]","[0.8392370572207085, 0.7322239031770045]","[541, 340]"
0,logreg,79.80,79.23,2.50,"[0.827027027027027, 0.7484662576687117]","[0.8484288354898336, 0.7176470588235294]","[0.8375912408759125, 0.7327327327327327]","[541, 340]"
4,perceptron,79.80,67.86,16.88,"[0.8318098720292505, 0.7425149700598802]","[0.8410351201478743, 0.7294117647058823]","[0.8363970588235293, 0.7359050445103856]","[541, 340]"


In [39]:
scores_df.sort_values(by='CV_Mean', ascending=False,inplace=True)
scores_df.head(100)

,Model,Score,CV_Mean,CV_SDEV,Precision,recall,fscore,support
10,random_forest,92.74,82.29,2.19,"[0.9176882661996497, 0.9451612903225807]","[0.9685767097966729, 0.861764705882353]","[0.9424460431654677, 0.9015384615384615]","[541, 340]"
2,svc,86.72,81.39,6.09,"[0.8693379790940766, 0.8631921824104235]","[0.922365988909427, 0.7794117647058824]","[0.895067264573991, 0.8191653786707882]","[541, 340]"
9,decision_tree,92.74,80.82,3.18,"[0.9090909090909091, 0.9630872483221476]","[0.9796672828096118, 0.8441176470588235]","[0.9430604982206405, 0.8996865203761755]","[541, 340]"
5,mlp,82.52,80.71,4.62,"[0.8230383973288815, 0.8297872340425532]","[0.911275415896488, 0.6882352941176471]","[0.8649122807017544, 0.7524115755627009]","[541, 340]"
1,ridge,80.02,79.91,2.77,"[0.8288288288288288, 0.7515337423312883]","[0.8502772643253235, 0.7205882352941176]","[0.8394160583941606, 0.7357357357357357]","[541, 340]"
7,linear_svc,80.02,79.80,2.70,"[0.8288288288288288, 0.7515337423312883]","[0.8502772643253235, 0.7205882352941176]","[0.8394160583941606, 0.7357357357357357]","[541, 340]"
0,logreg,79.80,79.23,2.50,"[0.827027027027027, 0.7484662576687117]","[0.8484288354898336, 0.7176470588235294]","[0.8375912408759125, 0.7327327327327327]","[541, 340]"
6,gaussian,78.66,78.10,3.69,"[0.8400770712909441, 0.7099447513812155]","[0.8059149722735675, 0.7558823529411764]","[0.8226415094339622, 0.7321937321937322]","[541, 340]"
3,knn,87.29,77.76,3.20,"[0.8921389396709324, 0.8413173652694611]","[0.9020332717190388, 0.8264705882352941]","[0.8970588235294117, 0.8338278931750741]","[541, 340]"
4,perceptron,79.80,67.86,16.88,"[0.8318098720292505, 0.7425149700598802]","[0.8410351201478743, 0.7294117647058823]","[0.8363970588235293, 0.7359050445103856]","[541, 340]"


In [40]:
# Random Forest is the best algo to use for prediction
Y_pred = random_forest.predict(X_test)
X_test = test_df

submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })


In [41]:
#create submission file
#submission.to_csv('/home/mathewlech/sandbox_local/output/titanic/submission.csv', index=False)
#submission.head(500)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0
